## G-CARE Benchmark

In [8]:
using Graphs: DiGraph
using StatsPlots
include("Source/datasets.jl")
include("Source/QuasiStableCardinalityEstimator.jl")


aids_data_file_path = "dataset/aids/aids.txt"
human_data_file_path = "dataset/human/human.txt"
lubm80_data_file_path = "dataset/lubm80/lubm80.txt"
yago_data_file_path = "dataset/yago/yago.txt"

#aids_data = load_dataset(aids_data_file_path)
#human_data = load_dataset(human_data_file_path)
#lubm80_data = load_dataset(lubm80_data_file_path)
yago_data = load_dataset(yago_data_file_path)
nothing

In [9]:
#datasets = [aids_data, human_data, lubm80_data, yago_data]
#dataset_names = ["aids", "human", "lubm80", "yago"]
datasets = [yago_data]
dataset_names = ["yago"]

1-element Vector{String}:
 "yago"

In [10]:
build_time = []
summary_size = []
color_summaries =[]
for dataset in datasets
    results = @timed generate_color_summary(dataset, 8, verbose=true)
    push!(build_time, results[2])
    push!(summary_size,  get_color_summary_size(results[1]))
    push!(color_summaries, results[1])
end

Started coloring


OutOfMemoryError: OutOfMemoryError()

In [5]:
lubm80_query_paths = ["lubm80/lubm80_Q2.txt",
"lubm80/lubm80_Q4.txt",
"lubm80/lubm80_Q7.txt",
"lubm80/lubm80_Q8.txt",
"lubm80/lubm80_Q9.txt",
"lubm80/lubm80_Q12.txt"
                    ]
exactSizes = []
bounds = []
relativeErrors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in lubm80_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[1])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./exact_size_results[1])
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
end

Summary Size: 

7449


Summary Build Time: 

563.481057


Query: lubm80/lubm80_Q2.txt
Bound Time: 

...

Query: lubm80/lubm80_Q8.txt


Bound Time: [0.0, 57.96121052020762, 4.575268e6]
Exact Size Time: 23.8088262
Exact Size: 7790
GCare Exact Size: 7790
Bound [Lower, Avg, Upper]: [0.0, 57.96121052020762, 4.575268e6]
Relative Error [Lower, Avg, Upper]: [0.0, 0.007440463481413045, 587.3258023106547]
Query: lubm80/lubm80_Q9.txt


Bound Time: [0.0, 0.0, 9.219986e6]
Exact Size Time: 21.7950568
Exact Size: 21872
GCare Exact Size: 21872
Bound [Lower, Avg, Upper]: [0.0, 0.0, 9.219986e6]
Relative Error [Lower, Avg, Upper]: [

0.0, 0.0, 421.54288588149234]
Query: lubm80/lubm80_Q12.txt


Bound Time: [0.0, 888.7787732598208, 145100.0]
Exact Size Time: 1.1935319
Exact Size: 15
GCare Exact Size: 15
Bound [Lower, Avg, Upper]: [0.0, 888.7787732598208, 145100.0]
Relative Error [Lower, Avg, Upper]: [0.0, 59.25191821732139, 9673.333333333334]


In [7]:
human_query_paths = ["human/Chain_3/uf_Q_1_1.txt",
"human/Chain_3/uf_Q_1_2.txt",
"human/Chain_3/uf_Q_1_3.txt",
"human/Star_3/uf_Q_0_1.txt",
"human/Star_3/uf_Q_0_2.txt",
"human/Star_3/uf_Q_1_1.txt",
"human/Star_3/uf_Q_1_2.txt",
"human/Star_3/uf_Q_1_3.txt",
"human/Star_3/uf_Q_1_4.txt", 
"human/Star_3/uf_Q_1_5.txt",
"human/Star_3/uf_Q_1_6.txt",
"human/Star_3/uf_Q_2_1.txt",
"human/Star_3/uf_Q_2_2.txt",
"human/Star_3/uf_Q_2_3.txt",
"human/Graph_3/uf_Q_3_1.txt",
"human/Graph_3/uf_Q_3_2.txt",
"human/Graph_3/uf_Q_3_3.txt",
"human/Graph_3/uf_Q_3_4.txt",
"human/Graph_3/uf_Q_3_5.txt",
"human/Graph_3/uf_Q_3_6.txt",
                    ]
exactSizes = []
bounds = []
relativeErrors = []
println("Summary Size: ", summary_size[2])
println("Summary Build Time: ", build_time[2])
for query_path in human_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[2])
    exact_size_results = @timed get_exact_size(query, datasets[2])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./exact_size_results[1])
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
end

Summary Size: 918288
Summary Build Time: 16.4347619


Query: human/Chain_3/uf_Q_1_1.txt
Bound Time: 0.0061073
Exact Size Time: 0.0010609
Exact Size: 72
GCare Exact Size: 72


Bound [Lower, Avg, Upper]: [0.0, 111.22824321181466, 32575.0]


Relative Error [Lower, Avg, Upper]: [0.0, 1.5448367112752035, 452.43055555555554]
Query: human/Chain_3/uf_Q_1_2.txt


Bound Time: 0.0072745
Exact Size Time: 0.2174464
Exact Size: 22
GCare Exact Size: 22
Bound [Lower, Avg, Upper]: [0.0, 1658.2185532313567, 121117.0]
Relative Error [Lower, Avg, Upper]: [0.0, 75.3735706014253, 5505.318181818182]
Query: human/Chain_3/uf_Q_1_3.txt
Bound Time: 0.0080442
Exact Size Time: 0.0017171
Exact Size: 10
GCare Exact Size: 10
Bound [Lower, Avg, Upper]: [0.0, 16.15672684458399, 7302.0]
Relative Error [Lower, Avg, Upper]: [0.0, 1.6156726844583988, 730.2]
Query: human/Star_3/uf_Q_0_1.txt
Bound Time: 0.0003408
Exact Size Time: 0.0006801
Exact Size: 1
GCare Exact Size: 1
Bound [Lower, Avg, Upper]: [

...


Bound Time: 0.1605094
Exact Size Time: 0.0666712
Exact Size: 9610
GCare Exact Size: 9610
Bound [Lower, Avg, Upper]: [0.0, 205.5186309614163, 188492.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.021385913731677034, 19.614151925078044]
Query: human/Graph_3/uf_Q_3_2.txt


Bound Time: 0.0141158
Exact Size Time: 0.0490071
Exact Size: 6875
GCare Exact Size: 6875
Bound [Lower, Avg, Upper]: [0.0, 606.8165831272471, 172488.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.08826423027305412, 25.089163636363637]
Query: human/Graph_3/uf_Q_3_3.txt
Bound Time: 0.0080508
Exact Size Time: 0.0282448
Exact Size: 2830
GCare Exact Size: 2830
Bound [Lower, Avg, Upper]: [0.0, 171.66966653674376, 79841.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.060660659553619704, 28.212367491166077]
Query: human/Graph_3/uf_Q_3_4.txt


Bound Time: 0.0108552
Exact Size Time: 0.0511957
Exact Size: 8588
GCare Exact Size: 8588
Bound [Lower, Avg, Upper]: [0.0, 1441.4202819381626, 114795.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.16784120656010276,

 13.36690731252911]
Query: human/Graph_3/uf_Q_3_5.txt
Bound Time: 0.011814
Exact Size Time: 0.0566442
Exact Size: 9103
GCare Exact Size: 9103
Bound [Lower, Avg, Upper]: [0.0, 1485.0196917434023, 132533.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.1631351962807209, 14.559266175985938]
Query: human/Graph_3/uf_Q_3_6.txt


Bound Time: 0.0021316
Exact Size Time: 0.0249467
Exact Size: 2796
GCare Exact Size: 2796
Bound [Lower, Avg, Upper]: [0.0, 125.44721324479778, 27675.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.044866671403718805, 9.898068669527897]


In [ ]:
yago_query_paths = ["yago/Chain_3/f_Q_1_1.txt",
"yago/Chain_3/f_Q_1_2.txt",
"yago/Chain_3/f_Q_1_3.txt",
"yago/Star_3/f_Q_0_1.txt",
"yago/Star_3/f_Q_0_2.txt",
"yago/Star_3/f_Q_1_1.txt",
"yago/Star_3/f_Q_1_2.txt",
"yago/Star_3/f_Q_1_3.txt",
"yago/Star_3/f_Q_1_4.txt", 
"yago/Star_3/f_Q_1_5.txt",
"yago/Star_3/f_Q_1_6.txt",
"yago/Star_3/f_Q_2_1.txt",
"yago/Star_3/f_Q_2_2.txt",
"yago/Star_3/f_Q_2_3.txt",
"yago/Graph_3/f_Q_3_1.txt",
"yago/Graph_3/f_Q_3_2.txt",
"yago/Graph_3/f_Q_3_3.txt",
"yago/Graph_3/f_Q_3_7.txt",
"yago/Graph_3/f_Q_3_8.txt",
"yago/Graph_3/f_Q_3_10.txt",
                    ]
exactSizes = []
bounds = []
relativeErrors = []
println("Summary Size: ", summary_size[2])
println("Summary Build Time: ", build_time[2])
for query_path in yago_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[2])
    exact_size_results = @timed get_exact_size(query, datasets[2])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./exact_size_results[1])
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
end

In [9]:
aids_query_paths = ["aids/Chain_3/uf_Q_5_1.txt",
"aids/Chain_3/uf_Q_5_2.txt",
"aids/Chain_3/uf_Q_5_3.txt",
"aids/Tree_3/uf_Q_2_5.txt",
"aids/Tree_3/uf_Q_4_4.txt",
"aids/Tree_6/uf_Q_4_4.txt",
"aids/Tree_6/uf_Q_5_6.txt",
"aids/Star_6/uf_Q_2_3.txt",
"aids/Star_6/uf_Q_1_1.txt",
"aids/Cycle_3/uf_Q_2_14.txt",
"aids/Cycle_3/uf_Q_3_14.txt",
"aids/Flower_6/uf_Q_3_6.txt",
"aids/Flower_6/uf_Q_5_3.txt",
"aids/Flower_9/uf_Q_3_3.txt"
]
exactSizes = []
bounds = []
relativeErrors = []
exactTimes = []
boundTimes = []
for query_path in aids_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./gcare_size)
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
    push!(exactTimes, exact_size_results[2])
    push!(boundTimes, bound_results[2])
end

Query: aids/Chain_3/uf_Q_5_1.txt


Bound Time: 0.0095019
Exact Size Time: 1.8756065
Exact Size: 120348
GCare Exact Size: 120348
Bound [Lower, Avg, Upper]: [0.0, 278506.84136686835, 9.587906924e9]
Relative Error [Lower, Avg, Upper]: [0.0, 2.3141792249714856, 79668.18662545286]
Query: aids/Chain_3/uf_Q_5_2.txt


Bound Time: 0.0564841
Exact Size Time: 1.4080133
Exact Size: 120348
GCare Exact Size: 120348
Bound [Lower, Avg, Upper]: [0.0, 278506.84136686847, 7.888712657e9]
Relative Error [Lower, Avg, Upper]: [0.0, 2.3141792249714865, 65549.1795210556]
Query: aids/Chain_3/uf_Q_5_3.txt


Bound Time: 0.008877
Exact Size Time: 1.7459571
Exact Size: 120348
GCare Exact Size: 120348
Bound [Lower, Avg, Upper]: [0.0, 278506.84136686835, 9.587906924e9]
Relative Error [Lower, Avg, Upper]: [0.0, 2.3141792249714856, 79668.18662545286]
Query: aids/Tree_3/uf_Q_2_5.txt


Bound Time: 0.0046892
Exact Size Time: 0.051786
Exact Size: 221
GCare Exact Size: 221
Bound [Lower, Avg, Upper]: [0.0, 152.36976773877308, 3.547527e6]
Relative Error [Lower, Avg, Upper]: 

...


Bound Time: 0.2715804
Exact Size Time: 1.0864293
Exact Size: 290
GCare Exact Size: 290
Bound [Lower, Avg, Upper]: [0.0, 0.05091974157860992, 6.7336846e7]
Relative Error [Lower, Avg, Upper]: [0.0, 0.00017558531578831007, 232196.02068965518]
Query: aids/Cycle_3/uf_Q_3_14.txt


Bound Time: 1.2601968
Exact Size Time: 3.1977892
Exact Size: 3240
GCare Exact Size: 3240
Bound [Lower, Avg, Upper]: [0.0, 0.31875066785742107, 3.59344051e8]
Relative Error [Lower, Avg, Upper]: [0.0, 9.837983575846329e-5, 110908.65771604938]
Query: aids/Flower_6/uf_Q_3_6.txt


Bound Time: 1.7880904
Exact Size Time: 1.1131221
Exact Size: 2898
GCare Exact Size: 2898
Bound [Lower, Avg, Upper]: [0.0, 1.349153285953711, 1.1367361114826e13]
Relative Error [Lower, Avg, Upper]: [0.0, 0.0004655463374581473, 3.92248485673775e9]
Query: aids/Flower_6/uf_Q_5_3.txt


Bound Time: 2.7389141
Exact Size Time: 2.555658
Exact Size: 231756
GCare Exact Size: 231756
Bound [Lower, Avg, Upper]: [0.0, 0.8787815221330144, 2.2472628081588e13]
Relative Error [Lower, Avg, Upper]: [0.0, 3.791839357483795e-6, 9.69667584942267e7]
Query: aids/Flower_9/uf_Q_3_3.txt


Bound Time: 4.5140935
Exact Size Time: 10.5460624
Exact Size: 4836
GCare Exact Size: 4836
Bound [Lower, Avg, Upper]: [0.0, 7.449587749517581, 1.8992961675776957e18]
Relative Error [Lower, Avg, Upper]: [0.0, 0.0015404441169391192, 3.9274114300614056e14]


In [6]:
relativeErrors

6-element Vector{Any}:
 [0.0, 0.0, 2037.801886792453]
 [0.0, 0.018128757995797183, 2.430169411764706e7]
 [0.0, 8578.183718779555, 317560.89552238805]
 [0.0, 0.007440463481413045, 587.3258023106547]
 [0.0, 0.0, 421.54288588149234]
 [0.0, 59.25191821732139, 9673.333333333334]

In [7]:
exactSizes

6-element Vector{Any}:
   212
    34
    67
  7790
 21872
    15